In [1]:
## Get some random module
!pip install nbastats
import nbastats.nbastats as nbastats
!pip install py-goldsberry
import goldsberry

In [19]:
goldsberry.player.demographics(201566).player_info()[0]['HEIGHT']

u'6-3'

In [2]:
## Import a bunch of modules that may be helpful
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
import requests

In [326]:
## Saving what we merged, checking the shape to see how many shots we didn't have
joined_shots.to_csv("joined_shots_2014.csv")
print joined_shots.shape,all_shots.shape,all_chart.shape

(200532, 40) (203590, 21) (205550, 22)


In [9]:
## function for getting the shot log of a player given his ID and the year
def shot_log(year,player_id):
    api_param = {'DateFrom' : '',
                    'DateTo' : '',
                    'GameSegment': '',
                    'LastNGames': 0,
                    'LeagueID': "00",
                    'Location': '',
                    'Month': 0,
                    'OpponentTeamID': 0,
                    'Outcome': '',
                    'Period': 0,
                    'PlayerID': player_id,
                    'Season': year,
                    'SeasonSegment': '',
                    'SeasonType': "Regular Season",
                    'TeamID': 0,
                    'VsConference':'',
                    'VsDivision': ''
                               }
    pull=requests.get("http://stats.nba.com/stats/playerdashptshotlog?",params=api_param)
    headers = pull.json()['resultSets'][0]['headers']
    values = pull.json()['resultSets'][0]['rowSet']
    huh=pd.DataFrame([dict(zip(headers, value)) for value in values])
    return huh


In [3]:
# get players in season a
def players(a):
    url = "http://stats.nba.com/stats/commonallplayers?"
    api_param = {'IsOnlyCurrentSeason':"0",
                          'LeagueID': '00',
                          'Season': '2015-16'}
    pull = requests.get(url, params=api_param)
    _headers = pull.json()['resultSets'][0]['headers']
    _values = pull.json()['resultSets'][0]['rowSet']
    season=pd.DataFrame([dict(zip(_headers, value)) for value in _values])
    season = season.query("FROM_YEAR <= @a and TO_YEAR >= @a")
    return season

In [4]:
players2014=players('2014')
players2013=players('2013')

In [5]:
height_2013 = []
for indx,row in players2013.iterrows():
    height_2013.append(goldsberry.player.demographics(row['PERSON_ID']).player_info()[0]['HEIGHT'])

In [6]:
len(height_2013),len(players2013.index)

(505, 505)

In [7]:
players2013['HEIGHT'] = height_2013

In [8]:
height_2014 = []
for indx,row in players2014.iterrows():
    height_2014.append(goldsberry.player.demographics(row['PERSON_ID']).player_info()[0]['HEIGHT'])
players2014['HEIGHT'] = height_2014   

In [293]:
#Get shot chart and shot log for all players in 2014
#Don't run, already been saved
all_shots= pd.DataFrame()
all_chart = pd.DataFrame()
for index, row in players2014.iterrows():
    chart = nbastats.ShotChart(row["PERSON_ID"],season="2014-15")
    all_chart=all_chart.append(chart.shotchart())
    shots = shot_log("2014-15",row["PERSON_ID"])
    shots['PLAYER_ID']=row["PERSON_ID"]
    all_shots = all_shots.append(shots)
all_shots.to_csv("all_shots_2014.csv")
all_chart.to_csv("all_chart_2014.csv")

In [295]:
#Get shot chart and shot log for all players in 2013
#Don't run, already been saved
all_shots= pd.DataFrame()
all_chart = pd.DataFrame()
for index, row in season2013.iterrows():
    shots = shot_log("2013-14",row["PERSON_ID"])
    shots['PLAYER_ID']=row["PERSON_ID"]
    all_shots = all_shots.append(shots)
    chart = nbastats.ShotChart(row["PERSON_ID"],season="2013-14")
    all_chart=all_chart.append(chart.shotchart())
all_shots.to_csv("all_shots_2013.csv")
all_chart.to_csv("all_chart_2013.csv")

In [5]:
## Merging shot chart data with player tracking for 2014
## DO NOT RUN... takes a while
all_shots = pd.read_csv("all_shots_2013.csv")
all_chart = pd.read_csv("all_chart_2013.csv")
players=set(all_shots['PLAYER_ID'])
joined_shots=pd.DataFrame()
joined_shots_1=pd.DataFrame()
for player in players:
    shots = all_shots.loc[all_shots.PLAYER_ID==player].reset_index(drop=True)
    chart = all_chart.loc[all_chart.PLAYER_ID==player].reset_index(drop=True)
    split_time = shots.GAME_CLOCK.str.split(':')
    col_names=[u'CLOSEST_DEFENDER', u'CLOSEST_DEFENDER_PLAYER_ID', u'CLOSE_DEF_DIST', 
               u'DRIBBLES', u'FGM', u'FINAL_MARGIN', u'GAME_CLOCK', u'GAME_ID', u'LOCATION', 
               u'MATCHUP', u'PERIOD', u'PTS', u'PTS_TYPE', u'SHOT_CLOCK', u'SHOT_DIST', 
               u'SHOT_NUMBER', u'SHOT_RESULT', u'TOUCH_TIME', u'W',u'GRID_TYPE',u'GAME_EVENT_ID',
               u'PLAYER_ID', u'PLAYER_NAME', u'TEAM_ID',u'MINUTES_REMAINING', u'SECONDS_REMAINING', 
               u'EVENT_TYPE', u'ACTION_TYPE', u'SHOT_TYPE', u'SHOT_ZONE_BASIC', 
               u'SHOT_ZONE_AREA', u'SHOT_ZONE_RANGE', u'SHOT_DISTANCE', u'LOC_X', u'LOC_Y', 
               u'SHOT_ATTEMPTED_FLAG', u'SHOT_MADE_FLAG']
    time = pd.DataFrame(shots.GAME_CLOCK.str.split(':').tolist(), columns="MIN SECONDS".split())
    shots['MIN'] = time['MIN']
    shots['SECONDS'] = time['SECONDS']
    shots['TIME']=60*shots.MIN.astype(int)+shots.SECONDS.astype(int)
    chart['TIME']=60*chart.MINUTES_REMAINING+chart.SECONDS_REMAINING
    games = set(chart["GAME_ID"])
    for game in games:
        game_shots_chart=chart.loc[chart.GAME_ID==game].reset_index(drop=True)
        game_shots_log = shots.loc[shots.GAME_ID==game].reset_index(drop=True)
        if game_shots_chart.shape[0]==game_shots_log.shape[0]:
            huh = pd.concat([game_shots_chart,game_shots_log],axis=1)
            huh=huh[col_names]
            joined_shots=joined_shots.append(huh)
        else:
            quarters=set(game_shots_chart['PERIOD'])
            for quarter in quarters:
                same_period=game_shots_log.loc[(game_shots_log.PERIOD==quarter)].reset_index(drop=True)
                same_period_chart=game_shots_chart.loc[(game_shots_chart.PERIOD==quarter)].reset_index(drop=True)
                if (not same_period.empty) and (not same_period_chart.empty):
                    if same_period_chart.shape[0]==same_period.shape[0]:
                        huh = pd.concat([same_period_chart,same_period],axis=1)
                        huh=huh[col_names]
                        joined_shots=joined_shots.append(huh)
                    else:
                        for index,row in same_period_chart.iterrows():
                            same_period['DIF']=abs(same_period.TIME-row.TIME)
                            df=same_period.sort(['DIF'],ascending=True)
                            same_time=same_period.loc[[0]]
                            if same_time.DIF[0]<5:
                                a=same_period_chart.loc[[index]].reset_index(drop=True)
                                huh=pd.concat([a,same_time],axis=1)
                                huh=huh[col_names]
                                joined_shots_1=joined_shots_1.append(huh,ignore_index=True)
    print player
joined_shots

202779
200745
200746
200751
200752
200755
200756
200757
200758
200761
2399
200765
200768
200769
200770
200777
200779
200780
200782
200794
200797
2405
2406
200811
203112
200826
200839
203458
203459
101107
2199
2200
203460
2203
2207
2210
2215
2216
2222
2223
2224
2225
2229
202952
202962
202970
203471
203476
101108
203144
2365
203076
203077
203078
203079
203080
203081
203082
203083
203084
203085
203086
203087
203088
203089
203090
203091
203092
203093
203094
203095
203096
203098
203099
203100
203101
203102
203103
2400
203106
2403
203108
203109
203110
203111
2408
203113
203114
203115
203116
203117
203118
203120
203121
2419
203124
2422
203129
203130
2427
203133
2430
203135
203136
203138
203139
203141
203142
203143
2440
203145
203146
203148
2446
2449
203156
2457
203506
203183
203187
201141
201142
201143
201144
201145
201147
201148
201149
201150
201152
201155
201156
2501
201158
201160
201162
201163
201166
201167
201168
201171
201175
201177
203515
201188
201189
201196
2544
201202
2547
2548
2549


,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,DRIBBLES,FGM,FINAL_MARGIN,GAME_CLOCK,GAME_ID,GAME_ID,LOCATION,MATCHUP,PERIOD,PERIOD,PTS,PTS_TYPE,SHOT_CLOCK,SHOT_DIST,SHOT_NUMBER,SHOT_RESULT,TOUCH_TIME,W,GRID_TYPE,GAME_EVENT_ID,PLAYER_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG
0,"Turner, Evan",202323,1.9,17,0,10,8:09,21300165,21300165,A,"NOV 20, 2013 - TOR @ PHI",2,2,0,2,9.8,7.7,1,missed,13.5,W,Shot Chart Detail,163,202779,202779,Dwight Buycks,1610612761,8,6,Missed Shot,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,6,-40,53,1,0
1,"Turner, Evan",202323,3.2,0,1,10,9:10,21300165,21300165,A,"NOV 20, 2013 - TOR @ PHI",4,4,3,3,8.1,23.5,2,made,1.4,W,Shot Chart Detail,449,202779,202779,Dwight Buycks,1610612761,9,7,Made Shot,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,26,182,195,1,1
2,"Carter-Williams, Michael",203487,3.5,0,0,10,8:36,21300165,21300165,A,"NOV 20, 2013 - TOR @ PHI",4,4,0,3,11.0,25.7,3,missed,1.3,W,Shot Chart Detail,452,202779,202779,Dwight Buycks,1610612761,8,33,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,26,159,206,1,0
3,"Brown, Lorenzo",203485,3.6,19,0,10,0:23,21300165,21300165,A,"NOV 20, 2013 - TOR @ PHI",4,4,0,3,9.0,24.7,4,missed,14.5,W,Shot Chart Detail,575,202779,202779,Dwight Buycks,1610612761,0,21,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Center(C),24+ ft.,27,-15,274,1,0
0,"Cole, Norris",202708,3.7,0,1,-7,11:43,21300230,21300230,H,"NOV 29, 2013 - TOR vs. MIA",2,2,3,3,NaN,25.2,1,made,1.0,L,Shot Chart Detail,134,202779,202779,Dwight Buycks,1610612761,11,41,Made Shot,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,24,-149,198,1,1
1,"Andersen, Chris",2365,1.6,13,0,-7,9:36,21300230,21300230,H,"NOV 29, 2013 - TOR vs. MIA",2,2,0,2,NaN,9.3,2,missed,9.3,L,Shot Chart Detail,146,202779,202779,Dwight Buycks,1610612761,9,34,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),8-16 ft.,9,-98,4,1,0
2,"Cole, Norris",202708,2.4,5,0,-7,8:21,21300230,21300230,H,"NOV 29, 2013 - TOR vs. MIA",2,2,0,3,2.6,28.4,3,missed,4.5,L,Shot Chart Detail,159,202779,202779,Dwight Buycks,1610612761,8,18,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,28,-100,268,1,0
0,"Jones, Terrence",203093,3.4,6,0,-6,10:48,21300103,21300103,A,"NOV 11, 2013 - TOR @ HOU",2,2,0,2,NaN,1.8,1,missed,4.5,L,Shot Chart Detail,160,202779,202779,Dwight Buycks,1610612761,10,48,Missed Shot,Jump Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,3,2,30,1,0
1,"Lin, Jeremy",202391,2.4,8,0,-6,10:30,21300103,21300103,A,"NOV 11, 2013 - TOR @ HOU",2,2,0,2,NaN,22.3,2,missed,6.8,L,Shot Chart Detail,201,202779,202779,Dwight Buycks,1610612761,10,28,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Right Side Center(RC),16-24 ft.,21,145,163,1,0
2,"Beverley, Patrick",201976,3.5,2,1,-6,8:26,21300103,21300103,A,"NOV 11, 2013 - TOR @ HOU",2,2,2,2,NaN,16.1,3,made,2.4,L,Shot Chart Detail,187,202779,202779,Dwight Buycks,1610612761,8,24,Made Shot,Pullup Jump shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-97,153,1,1


In [6]:
## Saving what we merged, checking the shape to see how many shots we didn't have
joined_shots.to_csv("joined_shots_2013.csv")
print joined_shots.shape,all_shots.shape,all_chart.shape

(200963, 40) (202689, 21) (204126, 22)


In [ ]:
##NEED TO DO THE SAME FOR 2013

In [345]:
joined_shots=pd.read_csv("joined_shots_2014.csv")
max(joined_shots.LOC_Y)
plt.scatter(joined_shots.LOC_X,joined_shots.LOC_Y)
plt.show()

In [9]:
all2013=pd.read_csv('joined_shots_2013.csv')

In [10]:
all2013['off_height']=0
all2013['def_height']=0

In [11]:
players_shot=list(set(all2013['PLAYER_ID']))
players_def=list(set(all2013['CLOSEST_DEFENDER_PLAYER_ID']))

In [71]:
at = all2013[all2013['PLAYER_ID']==2544].index.tolist()
all2013.loc[at]

,Unnamed: 0,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,DRIBBLES,FGM,FINAL_MARGIN,GAME_CLOCK,GAME_ID,GAME_ID.1,LOCATION,MATCHUP,PERIOD,PERIOD.1,PTS,PTS_TYPE,SHOT_CLOCK,SHOT_DIST,SHOT_NUMBER,SHOT_RESULT,TOUCH_TIME,W,GRID_TYPE,GAME_EVENT_ID,PLAYER_ID,PLAYER_ID.1,PLAYER_NAME,TEAM_ID,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,off_height,def_height
63215,0,"Crawford, Jamal",2037,4.1,2,1,4,9:27,21300736,21300736,A,"FEB 05, 2014 - MIA @ LAC",1,1,2,2,13.1,1.5,1,made,1.4,W,Shot Chart Detail,19,2544,2544,LeBron James,1610612748,9,26,Made Shot,Driving Dunk Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,0,22,1,1,6-8,0
63216,1,"Jordan, DeAndre",201599,5.5,0,1,4,7:38,21300736,21300736,A,"FEB 05, 2014 - MIA @ LAC",1,1,2,2,21.7,5.8,2,made,0.4,W,Shot Chart Detail,31,2544,2544,LeBron James,1610612748,7,37,Made Shot,Driving Dunk Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,3,21,22,1,1,6-8,0
63217,2,"Barnes, Matt",2440,1.4,3,1,4,4:57,21300736,21300736,A,"FEB 05, 2014 - MIA @ LAC",1,1,2,2,19.4,4.5,3,made,3.5,W,Shot Chart Detail,53,2544,2544,LeBron James,1610612748,4,55,Made Shot,Driving Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,13,7,1,1,6-8,0
63218,3,"Jordan, DeAndre",201599,6.2,11,0,4,1:23,21300736,21300736,A,"FEB 05, 2014 - MIA @ LAC",1,1,0,3,14.1,24.8,4,missed,9.6,W,Shot Chart Detail,82,2544,2544,LeBron James,1610612748,1,20,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,25,84,242,1,0,6-8,0
63219,4,"Griffin, Blake",201933,4.9,2,0,4,0:53,21300736,21300736,A,"FEB 05, 2014 - MIA @ LAC",1,1,0,2,3.0,21.7,5,missed,5.5,W,Shot Chart Detail,85,2544,2544,LeBron James,1610612748,0,50,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,19,-123,157,1,0,6-8,0
63220,5,"Griffin, Blake",201933,1.0,5,1,4,6:07,21300736,21300736,A,"FEB 05, 2014 - MIA @ LAC",2,2,2,2,7.0,3.7,6,made,5.1,W,Shot Chart Detail,188,2544,2544,LeBron James,1610612748,6,5,Made Shot,Driving Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,3,15,34,1,1,6-8,0
63221,6,"Barnes, Matt",2440,1.0,6,0,4,4:50,21300736,21300736,A,"FEB 05, 2014 - MIA @ LAC",2,2,0,2,18.5,6.8,7,missed,5.1,W,Shot Chart Detail,196,2544,2544,LeBron James,1610612748,4,55,Missed Shot,Turnaround Jump Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,-21,17,1,0,6-8,0
63222,7,"Barnes, Matt",2440,2.0,0,1,4,4:48,21300736,21300736,A,"FEB 05, 2014 - MIA @ LAC",2,2,2,2,24.0,2.9,8,made,0.1,W,Shot Chart Detail,198,2544,2544,LeBron James,1610612748,4,48,Made Shot,Driving Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,21,17,1,1,6-8,0
63223,8,"Barnes, Matt",2440,0.3,2,0,4,3:21,21300736,21300736,A,"FEB 05, 2014 - MIA @ LAC",2,2,0,2,3.8,3.7,9,missed,6.0,W,Shot Chart Detail,218,2544,2544,LeBron James,1610612748,3,19,Missed Shot,Floating Jump shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,5,34,37,1,0,6-8,0
63224,9,"Barnes, Matt",2440,7.4,0,1,4,1:37,21300736,21300736,A,"FEB 05, 2014 - MIA @ LAC",2,2,3,3,12.7,23.6,10,made,0.6,W,Shot Chart Detail,239,2544,2544,LeBron James,1610612748,1,34,Made Shot,Jump Shot,3PT Field Goal,Left Corner 3,Left Side(L),24+ ft.,24,-245,6,1,1,6-8,0


In [135]:
hi = players2013.loc[players2013.PERSON_ID==203111]
hi

,DISPLAY_LAST_COMMA_FIRST,FROM_YEAR,GAMES_PLAYED_FLAG,PERSON_ID,PLAYERCODE,ROSTERSTATUS,TEAM_ABBREVIATION,TEAM_CITY,TEAM_CODE,TEAM_ID,TEAM_NAME,TO_YEAR,HEIGHT
1827,"Johnson, Orlando",2012,Y,203111,orlando_johnson,0,,,,0,,2013,


In [12]:
for player in players_shot:
    at = all2013[all2013['PLAYER_ID']==player].index.tolist()
    hi = players2013.loc[players2013.PERSON_ID==player]['HEIGHT']
    all2013['off_height'][at]=hi.iloc[0]
for player in players_def:
    at = all2013[all2013['CLOSEST_DEFENDER_PLAYER_ID']==player].index.tolist()
    hi = players2013.loc[players2013.PERSON_ID==player]['HEIGHT']
    all2013['def_height'][at]=hi.iloc[0]

/Users/harrisonchase/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/harrisonchase/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [36]:
all2013[all2013['PLAYER_ID']==2544].index.tolist()
all2013['off_height'][[1,2]]

1    0
2    0
Name: off_height, dtype: int64

In [77]:
plt.scatter(all2013['SHOT_DIST'],all2013['SHOT_DISTANCE'],alpha=.2)
plt.show()

/Users/harrisonchase/anaconda/lib/python2.7/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [15]:
int(all2013.iloc[0]['PLAYER_ID'])
players2013.loc[players2013.PERSON_ID==all2013.iloc[0]['PLAYER_ID']]['PERSON_ID']

515    202779
Name: PERSON_ID, dtype: int64

In [ ]:
for index,row in all2013.iterrows():
    if index % 1000==0:
        print index
    all2013.iloc[index]['off_height']=players2013.loc[players2013.PERSON_ID==row['PLAYER_ID']]['HEIGHT']

In [116]:
### Merge play by play and shots
pbp2013=pd.read_csv('playbyplay/pbp_with_lineups_2013.csv')
sort=pbp2013.sort(['GAME_ID','EVENTNUM'])
huh = sort.drop_duplicates(['GAME_ID','EVENTNUM'])

In [117]:
huh1=[(x,y) for x,y in enumerate(huh.GAME_ID)]
huh2=huh1[1:]
start = [0]+[index for index,x in huh2 if x-huh1[index-1][1]>0]

In [109]:
len(tie)

6667

In [118]:
huh['SCOREMARGIN'][start]=0
huh['SCOREMARGIN']=[0 if x=='TIE' else x for x in huh.SCOREMARGIN]

/Users/harrisonchase/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [120]:
k=0
while (sum([(str(x) == 'nan') for x in huh.SCOREMARGIN])>0)&(k<50):
    k+=1
    print k
    huh['SCOREMARGIN']=[y if str(x) =='nan' else x for x,y in zip(list(huh.SCOREMARGIN),[0]+list(huh.SCOREMARGIN)[:-1])]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40


In [121]:
sum([(str(x) == 'TIE') for x in huh.SCOREMARGIN])


0

In [86]:
[y if str(x) =='nan' else x for x,y in zip(list(huh.SCOREMARGIN),[0]+list(huh.SCOREMARGIN)[:-1])]

[0,
 0,
 0,
 0,
 0,
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '5',
 '5',
 '5',
 '5',
 '5',
 '7',
 '7',
 '7',
 '7',
 '7',
 '7',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '12',
 '12',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '10',
 '7',
 '7',
 '7',
 '7',
 '9',
 '9',
 '9',
 '9',
 '10',
 '8',
 '8',
 '8',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '6',
 '3',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '6',
 '6',
 '6',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '5',
 '5',
 '5',
 '3',
 '3',
 '3',
 '5',
 '5',
 '5',
 '5',
 '5',
 '3',
 '3',
 '3',
 '3',
 '3',
 '5',
 '5',
 '4',
 '3',
 '3',
 '3',
 '3',
 '3',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '7',
 '4',
 '4',
 '4',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 'TIE',
 'TIE',
 'TIE',
 'TIE',
 'TIE',
 '-2',
 '1',
 '-1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '

In [ ]:
huh['score_margin']=0
for index,row in huh.iterrows():
    if index==0:
        huh.iloc[index]['score_margin']=0
    elif str(row['SCOREMARGIN'])=='nan':
        huh.iloc[index]['score_margin'] = huh.iloc[index-1]['score_margin']
    else:
        huh.iloc[index]['score_margin'] = huh.iloc[index]['SCOREMARGIN']
    if index % 1000==0:
        print index

/Users/harrisonchase/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/harrisonchase/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/harrisonchase/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-c

0
1000

In [115]:
len(["nan"]+list(huh.EVENTMSGTYPE))

540784

In [122]:
huh['ACTION_BEFORE']=["nan"]+list(huh.EVENTMSGTYPE)[:-1]
huh[1:30]

,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,NEUTRALDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,PERSON2TYPE,PERSON3TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,PLAYER1_TEAM_CITY,PLAYER1_TEAM_ID,PLAYER1_TEAM_NICKNAME,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ABBREVIATION,PLAYER2_TEAM_CITY,PLAYER2_TEAM_ID,PLAYER2_TEAM_NICKNAME,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,PLAYER3_TEAM_CITY,PLAYER3_TEAM_ID,PLAYER3_TEAM_NICKNAME,SCORE,SCOREMARGIN,VISITORDESCRIPTION,WCTIMESTRING,HOME_TEAM_ID,AWAY_TEAM_ID,HOME_PLAYER1_ID,HOME_PLAYER2_ID,HOME_PLAYER3_ID,HOME_PLAYER4_ID,HOME_PLAYER5_ID,AWAY_PLAYER1_ID,AWAY_PLAYER2_ID,AWAY_PLAYER3_ID,AWAY_PLAYER4_ID,AWAY_PLAYER5_ID,ACTION_BEFORE
1,0,10,1,21300001,Jump Ball Hibbert vs. Vucevic: Tip to West,NaN,12:00,1,4,5,4,201579,Roy Hibbert,IND,Indiana,1610612754,Pacers,202696,Nikola Vucevic,ORL,Orlando,1610612753,Magic,2561,David West,IND,Indiana,1610612754,Pacers,NaN,0,NaN,7:12 PM,1610612754,1610612753,2561,201579,201588,202331,202362,2749,101131,201167,202696,203090,12
2,1,2,2,21300001,MISS West 6' Jump Shot,NaN,11:39,1,4,0,0,2561,David West,IND,Indiana,1610612754,Pacers,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,7:12 PM,1610612754,1610612753,2561,201579,201588,202331,202362,2749,101131,201167,202696,203090,10
3,0,4,3,21300001,Hibbert REBOUND (Off:1 Def:0),NaN,11:39,1,4,0,0,201579,Roy Hibbert,IND,Indiana,1610612754,Pacers,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,7:12 PM,1610612754,1610612753,2561,201579,201588,202331,202362,2749,101131,201167,202696,203090,2
4,3,6,4,21300001,NaN,NaN,11:39,1,5,4,1,202696,Nikola Vucevic,ORL,Orlando,1610612753,Magic,201579,Roy Hibbert,IND,Indiana,1610612754,Pacers,0,NaN,NaN,NaN,NaN,NaN,NaN,0,Vucevic L.B.FOUL (P1.T1),7:12 PM,1610612754,1610612753,2561,201579,201588,202331,202362,2749,101131,201167,202696,203090,4
5,79,1,5,21300001,George 19' Pullup Jump Shot (2 PTS),NaN,11:30,1,4,0,0,202331,Paul George,IND,Indiana,1610612754,Pacers,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0 - 2,2,NaN,7:13 PM,1610612754,1610612753,2561,201579,201588,202331,202362,2749,101131,201167,202696,203090,6
6,1,2,6,21300001,NaN,NaN,11:15,1,5,0,0,201167,Arron Afflalo,ORL,Orlando,1610612753,Magic,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,2,MISS Afflalo 27' 3PT Jump Shot,7:13 PM,1610612754,1610612753,2561,201579,201588,202331,202362,2749,101131,201167,202696,203090,1
7,0,4,7,21300001,Hibbert REBOUND (Off:1 Def:1),NaN,11:14,1,4,0,0,201579,Roy Hibbert,IND,Indiana,1610612754,Pacers,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,7:13 PM,1610612754,1610612753,2561,201579,201588,202331,202362,2749,101131,201167,202696,203090,2
8,5,2,8,21300001,MISS West 3' Layup,NaN,11:03,1,4,0,5,2561,David West,IND,Indiana,1610612754,Pacers,0,NaN,NaN,NaN,NaN,NaN,101131,Jason Maxiell,ORL,Orlando,1610612753,Magic,NaN,2,Maxiell BLOCK (1 BLK),7:13 PM,1610612754,1610612753,2561,201579,201588,202331,202362,2749,101131,201167,202696,203090,4
9,0,4,9,21300001,NaN,NaN,11:02,1,5,0,0,101131,Jason Maxiell,ORL,Orlando,1610612753,Magic,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,2,Maxiell REBOUND (Off:0 Def:1),7:13 PM,1610612754,1610612753,2561,201579,201588,202331,202362,2749,101131,201167,202696,203090,2
10,1,5,10,21300001,West STEAL (1 STL),NaN,11:01,1,5,4,0,101131,Jason Maxiell,ORL,Orlando,1610612753,Magic,2561,David West,IND,Indiana,1610612754,Pacers,0,NaN,NaN,NaN,NaN,NaN,NaN,2,Maxiell Bad Pass Turnover (P1.T1),7:13 PM,1610612754,1610612753,2561,201579,201588,202331,202362,2749,101131,201167,202696,203090,4


In [123]:
all2013['EVENTNUM']=all2013['GAME_EVENT_ID']

In [124]:
try1=huh.merge(all2013,on=['GAME_ID','EVENTNUM'])

In [127]:
try1['SCOREMARGIN']=[-1*int(y) if x =='A' else y for x,y in zip(try1.LOCATION,try1.SCOREMARGIN)]

In [128]:
try1.head()

,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,NEUTRALDESCRIPTION,PCTIMESTRING,PERIOD_x,PERSON1TYPE,PERSON2TYPE,PERSON3TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,PLAYER1_TEAM_CITY,PLAYER1_TEAM_ID,PLAYER1_TEAM_NICKNAME,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ABBREVIATION,PLAYER2_TEAM_CITY,PLAYER2_TEAM_ID,PLAYER2_TEAM_NICKNAME,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,PLAYER3_TEAM_CITY,PLAYER3_TEAM_ID,PLAYER3_TEAM_NICKNAME,SCORE,SCOREMARGIN,VISITORDESCRIPTION,WCTIMESTRING,HOME_TEAM_ID,AWAY_TEAM_ID,HOME_PLAYER1_ID,HOME_PLAYER2_ID,HOME_PLAYER3_ID,HOME_PLAYER4_ID,HOME_PLAYER5_ID,AWAY_PLAYER1_ID,AWAY_PLAYER2_ID,AWAY_PLAYER3_ID,AWAY_PLAYER4_ID,AWAY_PLAYER5_ID,ACTION_BEFORE,Unnamed: 0,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,DRIBBLES,FGM,FINAL_MARGIN,GAME_CLOCK,GAME_ID.1,LOCATION,MATCHUP,PERIOD_y,PERIOD.1,PTS,PTS_TYPE,SHOT_CLOCK,SHOT_DIST,SHOT_NUMBER,SHOT_RESULT,TOUCH_TIME,W,GRID_TYPE,GAME_EVENT_ID,PLAYER_ID,PLAYER_ID.1,PLAYER_NAME,TEAM_ID,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,off_height,def_height
0,1,2,2,21300001,MISS West 6' Jump Shot,NaN,11:39,1,4,0,0,2561,David West,IND,Indiana,1610612754,Pacers,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,7:12 PM,1610612754,1610612753,2561,201579,201588,202331,202362,2749,101131,201167,202696,203090,10,0,"Maxiell, Jason",101131,1.6,2,0,10,11:41,21300001,H,"OCT 29, 2013 - IND vs. ORL",1,1,0,2,6.7,7.4,1,missed,7.8,W,Shot Chart Detail,2,2561,2561,David West,1610612754,11,39,Missed Shot,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,5,-38,45,1,0,6-9,6-7
1,79,1,5,21300001,George 19' Pullup Jump Shot (2 PTS),NaN,11:30,1,4,0,0,202331,Paul George,IND,Indiana,1610612754,Pacers,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0 - 2,2,NaN,7:13 PM,1610612754,1610612753,2561,201579,201588,202331,202362,2749,101131,201167,202696,203090,6,0,"Harkless, Maurice",203090,4.1,1,1,10,11:32,21300001,H,"OCT 29, 2013 - IND vs. ORL",1,1,2,2,16.8,19.3,1,made,3.2,W,Shot Chart Detail,5,202331,202331,Paul George,1610612754,11,30,Made Shot,Pullup Jump shot,2PT Field Goal,Mid-Range,Right Side Center(RC),16-24 ft.,19,105,164,1,1,6-9,6-9
2,1,2,6,21300001,NaN,NaN,11:15,1,5,0,0,201167,Arron Afflalo,ORL,Orlando,1610612753,Magic,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,-2,MISS Afflalo 27' 3PT Jump Shot,7:13 PM,1610612754,1610612753,2561,201579,201588,202331,202362,2749,101131,201167,202696,203090,1,0,"George, Paul",202331,4.4,0,0,-10,11:18,21300001,A,"OCT 29, 2013 - ORL @ IND",1,1,0,3,13.0,24.5,1,missed,0.7,L,Shot Chart Detail,6,201167,201167,Arron Afflalo,1610612753,11,15,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Center(C),24+ ft.,27,51,266,1,0,6-5,6-9
3,5,2,8,21300001,MISS West 3' Layup,NaN,11:03,1,4,0,5,2561,David West,IND,Indiana,1610612754,Pacers,0,NaN,NaN,NaN,NaN,NaN,101131,Jason Maxiell,ORL,Orlando,1610612753,Magic,NaN,2,Maxiell BLOCK (1 BLK),7:13 PM,1610612754,1610612753,2561,201579,201588,202331,202362,2749,101131,201167,202696,203090,4,1,"Maxiell, Jason",101131,0.4,0,0,10,11:05,21300001,H,"OCT 29, 2013 - IND vs. ORL",1,1,0,2,13.8,1.4,2,missed,1.8,W,Shot Chart Detail,8,2561,2561,David West,1610612754,11,3,Missed Shot,Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,28,-5,1,0,6-9,6-7
4,1,1,11,21300001,Stephenson 26' 3PT Jump Shot (3 PTS) (West 1 AST),NaN,10:51,1,4,4,0,202362,Lance Stephenson,IND,Indiana,1610612754,Pacers,2561,David West,IND,Indiana,1610612754,Pacers,0,NaN,NaN,NaN,NaN,NaN,0 - 5,5,NaN,7:14 PM,1610612754,1610612753,2561,201579,201588,202331,202362,2749,101131,201167,202696,203090,5,0,"Nelson, Jameer",2749,5.7,0,1,10,10:53,21300001,H,"OCT 29, 2013 - IND vs. ORL",1,1,3,3,15.6,25.1,1,made,0.7,W,Shot Chart Detail,11,202362,202362,Lance Stephenson,1610612754,10,51,Made Shot,Jump Shot,3PT Field Goal,Above the Break 3,Center(C),24+ ft.,26,15,260,1,1,6-5,6-0


In [129]:
try1.to_csv('merged_shots/merged_shots_13.csv')

In [130]:
all2014=pd.read_csv('joined_shots_2014.csv')
all2014['off_height']=0
all2014['def_height']=0
players_shot=list(set(all2014['PLAYER_ID']))
players_def=list(set(all2014['CLOSEST_DEFENDER_PLAYER_ID']))
for player in players_shot:
    at = all2014[all2014['PLAYER_ID']==player].index.tolist()
    hi = players2014.loc[players2014.PERSON_ID==player]['HEIGHT']
    all2014['off_height'][at]=hi.iloc[0]
for player in players_def:
    at = all2014[all2014['CLOSEST_DEFENDER_PLAYER_ID']==player].index.tolist()
    hi = players2014.loc[players2014.PERSON_ID==player]['HEIGHT']
    if len(hi)>0:
        all2014['def_height'][at]=hi.iloc[0]

/Users/harrisonchase/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/harrisonchase/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [36]:
pbp2014=pd.read_csv('playbyplay/pbp_with_lineups_2014.csv')
sort=pbp2014.sort(['GAME_ID','EVENTNUM'])
all2014['EVENTNUM']=all2014['GAME_EVENT_ID']
huh = sort.drop_duplicates(['GAME_ID','EVENTNUM'])
k=0
while (sum([(str(x) == 'nan') for x in huh.SCOREMARGIN])>0)&(k<50):
    k+=1
    print k
    huh['SCOREMARGIN']=[y if str(x) =='nan' else x for x,y in zip(list(huh.SCOREMARGIN),[0]+list(huh.SCOREMARGIN)[:-1])]
huh['ACTION_BEFORE']=["nan"]+list(huh.EVENTMSGTYPE)[:-1]
huh['SCOREMARGIN']=[0 if x=='TIE' else x for x in huh.SCOREMARGIN]
try1=huh.merge(all2014,on=['GAME_ID','EVENTNUM'])
try1['SCOREMARGIN']=[-1*int(y) if x =='A' else y for x,y in zip(try1.LOCATION,try1.SCOREMARGIN)]
try1.to_csv('merged_shots/merged_shots_14.csv')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45

/Users/harrisonchase/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/harrisonchase/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [74]:
huh=pd.read_csv('merged_shots/merged_shots_13.csv')

In [75]:
huh.SCOREMARGIN

0           0
1           2
2         NaN
3           2
4           5
5         NaN
6         NaN
7           7
8         NaN
9           8
10          8
11         10
12        NaN
13         10
14         12
15        NaN
16        NaN
17         10
18        NaN
19          8
20          8
21        NaN
22         10
23        NaN
24          9
25        NaN
26          8
27        NaN
28        NaN
29        NaN
         ... 
200933    TIE
200934    NaN
200935    TIE
200936    NaN
200937      3
200938    NaN
200939    NaN
200940      1
200941    TIE
200942    NaN
200943    NaN
200944     -1
200945    NaN
200946     -3
200947    NaN
200948     -2
200949    TIE
200950    NaN
200951     -3
200952    NaN
200953    NaN
200954     -2
200955    NaN
200956     -2
200957    NaN
200958    NaN
200959    NaN
200960     -2
200961     -3
200962     -4
Name: SCOREMARGIN, dtype: object

In [43]:
try1=huh.merge(all2014,on=['GAME_ID','EVENTNUM'])

[0,
 '2',
 -2,
 '2',
 '5',
 -5,
 -5,
 '7',
 -7,
 '8',
 '8',
 '10',
 -10,
 '10',
 '12',
 -10,
 -10,
 '10',
 -8,
 '8',
 '8',
 -8,
 '10',
 -7,
 '9',
 -8,
 '8',
 -6,
 -6,
 -6,
 '6',
 '6',
 -6,
 -3,
 '5',
 -5,
 '5',
 -5,
 -6,
 -4,
 '4',
 '4',
 -3,
 '5',
 -5,
 -3,
 '3',
 -3,
 '5',
 '3',
 '5',
 -5,
 '7',
 -4,
 -2,
 '2',
 '2',
 0,
 2,
 '1',
 1,
 '1',
 -1,
 '1',
 1,
 '1',
 2,
 '-2',
 2,
 4,
 '-4',
 4,
 4,
 '-4',
 4,
 '-4',
 4,
 '-2',
 2,
 0,
 0,
 2,
 2,
 2,
 4,
 '-4',
 4,
 4,
 3,
 1,
 1,
 '1',
 '3',
 -1,
 '1',
 -1,
 -1,
 -2,
 1,
 -1,
 '3',
 -3,
 '6',
 -3,
 -1,
 '3',
 -3,
 -3,
 1,
 '1',
 '4',
 -2,
 '2',
 -1,
 -3,
 -3,
 '3',
 -3,
 -1,
 '5',
 '7',
 -7,
 '7',
 '8',
 '10',
 -11,
 '13',
 -13,
 '15',
 -15,
 -13,
 '16',
 -16,
 '18',
 '18',
 -16,
 -19,
 '19',
 -19,
 '19',
 -19,
 -19,
 -17,
 -15,
 -15,
 -15,
 '17',
 -17,
 -15,
 '15',
 -12,
 '12',
 -10,
 -11,
 -11,
 -13,
 '13',
 -10,
 '2',
 -2,
 '2',
 '2',
 0,
 0,
 0,
 3,
 '-3',
 '-5',
 5,
 '-5',
 7,
 '-7',
 7,
 '-7',
 '-5',
 5,
 '-3',
 5,
 '-3',
 3,
 3,


In [70]:
[index for index,x in enumerate(try1.SCOREMARGIN) if x=='TIE']

[16,
 19,
 49,
 58,
 59,
 60,
 66,
 67,
 185,
 189,
 190,
 191,
 192,
 193,
 199,
 221,
 222,
 224,
 225,
 271,
 272,
 286,
 321,
 482,
 487,
 489,
 491,
 492,
 493,
 500,
 511,
 512,
 513,
 514,
 515,
 516,
 517,
 642,
 643,
 644,
 647,
 648,
 649,
 651,
 655,
 656,
 662,
 724,
 725,
 726,
 727,
 729,
 738,
 739,
 831,
 832,
 999,
 1000,
 1007,
 1011,
 1012,
 1013,
 1014,
 1017,
 1018,
 1022,
 1025,
 1026,
 1027,
 1045,
 1048,
 1066,
 1070,
 1071,
 1083,
 1087,
 1097,
 1102,
 1106,
 1107,
 1108,
 1109,
 1110,
 1111,
 1112,
 1178,
 1179,
 1324,
 1325,
 1326,
 1332,
 1333,
 1356,
 1361,
 1467,
 1471,
 1472,
 1475,
 1477,
 1478,
 1479,
 1480,
 1481,
 1483,
 1484,
 1497,
 1650,
 1660,
 1661,
 1669,
 1670,
 1671,
 1672,
 1673,
 1675,
 1689,
 1690,
 1691,
 1692,
 1693,
 1702,
 1723,
 1736,
 1737,
 1753,
 1819,
 1821,
 1823,
 1824,
 1825,
 1827,
 1829,
 1831,
 1842,
 1851,
 1852,
 1853,
 1854,
 1855,
 1856,
 1878,
 1880,
 1975,
 2128,
 2133,
 2134,
 2170,
 2175,
 2176,
 2178,
 2202,
 2203,
 